In [162]:
!pip install pandas nb_black plotly > /dev/null 2>&1

<IPython.core.display.Javascript object>

In [163]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [164]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from pprint import pprint

<IPython.core.display.Javascript object>

## Reference
![](rate.png)

Styles used:

```
Background - #F3F7F7  
Title - #3A3F4A, 36px  
Caption - #5D646F, 12px  
Axis text - #5D646F, 12 px  
Legend title & text - #5D646F, 12 px  
Font Family - Ubuntu Mono  
Color scale #67001f, #f7f7f7, #053061  
```

## Reproduction

In [165]:
df = pd.read_csv("rate.csv")
df = df.set_index(["region"])

regions = df.index.unique().tolist()
regions_by_rate = sorted(regions, key=lambda x: df.xs(x)["rate"].mean())
df.head()

,year,rate
region,,
Crimea,1990,2.5
Vinnytsia,1990,-2.1
Volyn,1990,4.0
Dnipropetrovsk,1990,0.5
Donetsk,1990,-1.2


<IPython.core.display.Javascript object>

In [166]:
def hex_to_RGB(hex_value):
    """ "#FFFFFF" -> [255,255,255] """
    # Pass 16 to the integer function for change of base
    return [int(hex_value[i : i + 2], 16) for i in range(1, 6, 2)]


def RGB_to_hex(RGB):
    """ [255,255,255] -> "#FFFFFF" """
    # Components need to be integers for hex to make sense
    RGB = [int(x) for x in RGB]
    return "#" + "".join(
        ["0{0:x}".format(v) if v < 16 else "{0:x}".format(v) for v in RGB]
    )


def linear_gradient(start_hex, finish_hex="#FFFFFF", n=10):
    """ returns a gradient list of (n) colors between
    two hex colors. start_hex and finish_hex
    should be the full six-digit color string,
    inlcuding the number sign ("#FFFFFF") """
    # Starting and ending colors in RGB form
    s = hex_to_RGB(start_hex)
    f = hex_to_RGB(finish_hex)
    # Initilize a list of the output colors with the starting color
    RGB_list = [s]
    # Calcuate a color at each evenly spaced value of t from 1 to n
    for t in range(1, n):
        # Interpolate RGB vector for color at the current value of t
        curr_vector = [
            int(s[j] + (float(t) / (n - 1)) * (f[j] - s[j])) for j in range(3)
        ]
        # Add it to our list of output colors
        RGB_list.append(curr_vector)
    return [RGB_to_hex(RGB) for RGB in RGB_list]


def discrete_colorscale(bvals, colors):
    """
    bvals - list of values bounding intervals/ranges of interest
    colors - list of rgb or hex colorcodes for values in [bvals[k], bvals[k+1]],0<=k < len(bvals)-1
    returns the plotly  discrete colorscale
    """
    if len(bvals) != len(colors) + 1:
        raise ValueError("len(boundary values) should be equal to  len(colors)+1")
    bvals = sorted(bvals)
    nvals = [
        (v - bvals[0]) / (bvals[-1] - bvals[0]) for v in bvals
    ]  # normalized values

    dcolorscale = []  # discrete colorscale
    for k in range(len(colors)):
        dcolorscale.extend([[nvals[k], colors[k]], [nvals[k + 1], colors[k]]])
    return dcolorscale

<IPython.core.display.Javascript object>

In [157]:
heatmap_granularity = 250
color_scale_steps = np.linspace(0, 1, num=heatmap_granularity)
heatmap_red_ratio = int(15 / 24 * granularity)

heatmap_colors = (
    linear_gradient("#67001f", "#f7f7f7", heatmap_red_ratio)[:-1]
    + ["#f7f7f7"]
    + linear_gradient("#f7f7f7", "#7F87A3", heatmap_granularity - heatmap_red_ratio)[1:]
)
colorspace = discrete_colorscale(color_scale_steps, heatmap_colors)

<IPython.core.display.Javascript object>

In [158]:
"""
Plotly doesn't support horizontal colorbar,
so I was forced to use Bar chart to emulate color bar 😞
"""


def create_bar(names: list, colors: list):
    fig = go.Figure()

    traces = []
    for i, color in zip(names, colors):
        traces.append(
            go.Bar(
                y=[""],
                x=[1],
                text=[i],
                width=0.1,
                orientation="h",
                marker_color=color,
                hovertemplate="%{text}",
                marker=dict(color=color, line=dict(color=color, width=0,),),
                xaxis="x2",
                yaxis="y2",
            )
        )
    return traces


colorbar_colors = (
    linear_gradient("#67001f", "#f7f7f7", 16)[:-1]
    + ["#f7f7f7"]
    + linear_gradient("#f7f7f7", "#7F87A3", 9)[1:]
)

names = list(range(-15, 1)) + list(range(1, 9))
bar = create_bar(names, colorbar_colors)

<IPython.core.display.Javascript object>

In [159]:
def generate_colorbar_ticks(text: list, position: list):
    return go.layout.Annotation(
        x=position[0],
        y=position[1],
        showarrow=False,
        text=text,
        xref="paper",
        yref="paper",
        font={"family": "Ubuntu Mono", "size": 12, "color": "#5D646F"},
    )


y_coord = 0.915
colorbar_ticks_coords = [0, 0.281, 0.613, 0.936]
texts = [-15, -8, 0, 8]
colorbar_ticks = [
    generate_colorbar_ticks(i, (x_coord, y_coord))
    for i, x_coord in zip(texts, colorbar_ticks_coords)
]

<IPython.core.display.Javascript object>

In [161]:
x = df["year"].unique()
# dirty hack to add offset to labels
y = [f" {r} " for r in regions_by_rate]
z = [df.xs(region)["rate"] for region in regions_by_rate]

heatmap = go.Heatmap(
    x=x,
    y=y,
    z=z,
    zmin=-15,
    zmax=8,
    colorscale=colorspace,
    showscale=False,
    colorbar=dict(
        x=-0.1,
        y=0.49,
        len=1.02,
        thickness=20,
        tickmode="array",
        tickvals=[-14.5, -7.8, -0.1, 7.5],
        ticktext=[-15, -8, 0, 8],
    ),
)

layout = go.Layout(
    title=go.layout.Title(
        text="<b>Population growth by region</b>",
        font={"family": "Ubuntu Mono", "size": 36, "color": "#3A3F4A"},
        x=0.05,
        y=0.905,
    ),
    margin={"r": 160, "t": 95, "l": 50},
    plot_bgcolor="#F3F7F7",
    paper_bgcolor="#F3F7F7",
    width=1000,
    height=700,
    font={"family": "Ubuntu Mono"},
    barmode="stack",
    showlegend=False,
    xaxis=dict(
        showgrid=False,
        side="top",
        tickfont={"family": "Ubuntu Mono", "color": "#5D646F"},
    ),
    yaxis=dict(
        domain=[0, 0.85],
        position=1,
        showgrid=False,
        side="right",
        tickfont={"family": "Ubuntu Mono", "color": "#5D646F"},
    ),
    xaxis2=dict(showgrid=False, showticklabels=False, zeroline=False),
    yaxis2=dict(domain=[0.85, 1], showgrid=False, showticklabels=False, zeroline=False),
    annotations=[
        *colorbar_ticks,
        go.layout.Annotation(
            x=1,
            y=-0.07,
            showarrow=False,
            text="Data: State Statistics Service of Ukraine",
            xref="paper",
            yref="paper",
            font={"family": "Ubuntu Mono", "size": 12, "color": "#5D646F"},
        ),
        go.layout.Annotation(
            x=0,
            y=0.97,
            showarrow=False,
            text="population growth rate, per 1000",
            xref="paper",
            yref="paper",
            font={"family": "Ubuntu Mono", "size": 12, "color": "#5D646F"},
        ),
    ],
)


fig = go.Figure(data=[heatmap, *bar], layout=layout)
fig.show()

<IPython.core.display.Javascript object>

![](rate.png)